# Matcha Helper
### 2026 Made with ❤️, Gilang Wahyu Prasetyo

# IMPORT

In [1]:
import os
import re
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
pd.set_option("display.max_columns", None)



# BUKA CHROME & REDIRECT

In [2]:
url = "https://matchapro.web.bps.go.id/dirgc"

chrome_cust = "isi di sini path nya"

user_agent = (
    "Mozilla/5.0 (Linux; Android 15; SM-G991B Build/TP1A.220624.014; wv) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 "
    "Chrome/132.0.6834.88 Mobile Safari/537.36"
)

options = Options()
options.add_argument("--start-maximized")
options.add_argument(f"--user-agent={user_agent}")

try:
    service = Service(executable_path=chrome_cust)
    driver = webdriver.Chrome(service=service, options=options)
except Exception:
    service = Service()
    driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

# LOGIN SSO BPS

In [3]:
username_sso = "sso"
password_sso = "pass"

wait = WebDriverWait(driver, 20)

# Klik tombol SSO
wait.until(EC.element_to_be_clickable((By.ID, "login-sso"))).click()

# Isi kredensial
wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username_sso)
driver.find_element(By.ID, "password").send_keys(password_sso)

# Submit login
driver.find_element(By.ID, "kc-login").click()

# Import Spreadsheet

In [10]:
# spreadsheet
sbrdup = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/isi_sendiri_link_spreadsheetmu/export?format=csv&gid=0",
    dtype={'latitude': str, 'longitude': str}  # Import dulu as string
)
#filter untuk keep atau remove
#mask = sbrdup["Petugas"].astype(str).str.lower().str.contains(
#    "dessy|karina|gilang|regita|adytia|nurul", 
#    case=False, 
#    na=False, 
#    regex=True
#)

# mask = sbrdup['nomor'].isna() | (sbrdup['nomor'].astype(str).str.strip() == '')
# filter keep
#sbrdup = sbrdup[mask].reset_index(drop=True)
# filter remove
#sbrdup = sbrdup[~mask].reset_index(drop=True)

sbrdup["idsbr"] = (
    sbrdup["idsbr"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .replace("nan", pd.NA)
    .astype("Int64")  # integer nullable
)
sbrdup = sbrdup.iloc[0:13000].reset_index(drop=True)
print("jumlah baris:", len(sbrdup))

jumlah baris: 13000


# Fix Latlong

In [5]:
# Fungsi untuk perbaiki format latlong dari text
def fix_latlong_format(value):
    if pd.isna(value):
        return None
    
    s = str(value).strip()
    if s == '' or s.lower() in ['nan', 'na', 'null']:
        return None
    
    # Ganti koma jadi titik (untuk locale Indonesia)
    s = s.replace(',', '.')
    
    # Hapus space/karakter aneh
    s = s.replace(' ', '')
    
    try:
        return float(s)
    except:
        return None

# Terapkan ke kolom latitude dan longitude
sbrdup["latitude"] = sbrdup["latitude"].apply(fix_latlong_format)
sbrdup["longitude"] = sbrdup["longitude"].apply(fix_latlong_format)

LAT_MIN, LAT_MAX = -2.358865559286161, -1.315090836134106
LON_MIN, LON_MAX = 115.13204822042515, 115.7495

def fix_latitude_decimal(lat):
    if pd.isna(lat):
        return lat
    
    # Kalau sudah dalam range Tabalong, skip
    if LAT_MIN <= lat <= LAT_MAX:
        return lat
    
    # Kalau positif atau terlalu besar/kecil, return as-is
    if lat >= 0 or lat < -10:
        return lat
    
    # Fix latitude yang salah tempat titiknya
    if -1 < lat < 0:  # antara 0 dan -1
        if lat > -0.01:  # -0.00xxx (geser 3 langkah)
            return lat * 1000
        elif lat > -0.1:  # -0.0xxx (geser 2 langkah)
            return lat * 100
        else:  # -0.xxx (geser 1 langkah)
            return lat * 10
    
    return lat

# Apply fix
sbrdup['latitude'] = sbrdup['latitude'].apply(fix_latitude_decimal)

# Filter (CUMA SEKALI)
mask_in = (
    sbrdup["latitude"].notna() &
    sbrdup["longitude"].notna() &
    sbrdup["latitude"].between(LAT_MIN, LAT_MAX, inclusive="both") &
    sbrdup["longitude"].between(LON_MIN, LON_MAX, inclusive="both")
)

# Simpan outlier
outlier = sbrdup.loc[~mask_in].copy()
#sbrdup.to_excel(r"C:\Users\ACER\Downloads\tambahan3453.xlsx", index=False)

# Keep yang di dalam bbox
#sbrdup = sbrdup.loc[mask_in].copy()

print("outlier:", len(outlier))
print("sisa:", len(sbrdup))
print(f"Latitude range: {sbrdup['latitude'].min():.6f} to {sbrdup['latitude'].max():.6f}")
print(f"Longitude range: {sbrdup['longitude'].min():.6f} to {sbrdup['longitude'].max():.6f}")
print(f"NaN lat: {sbrdup['latitude'].isna().sum()}")
print(f"NaN lon: {sbrdup['longitude'].isna().sum()}")


outlier: 0
sisa: 1528
Latitude range: -2.355427 to -1.679144
Longitude range: 115.242078 to 115.687077
NaN lat: 0
NaN lon: 0


In [ ]:
len(sbrdup)

# Isi Manual (Gilang Only)

In [ ]:
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(1)
# ======================
# 4. KLIK USAHA
# ======================

idsbr = str(sbrdup["idsbr"].iloc[4]).strip()
# 2) Isi IDSBR
inp = wait.until(EC.element_to_be_clickable((By.ID, "search-idsbr")))
inp.click()
inp.send_keys(Keys.CONTROL, "a")
inp.send_keys(Keys.BACKSPACE)
inp.send_keys(idsbr)
#inp.send_keys(Keys.ENTER)
time.sleep(5)

# 2.5) scroll jauh ke bawah
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)

# 3) Tunggu card pertama muncul (cukup ada headernya)
usaha = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.usaha-card")))
usaha.click()

# 3.5) scroll jauh ke bawah
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)

# 4) Klik tombol "Tandai" yang nempel ke CARD PERTAMA (refind saat klik)
btn_locator = (By.XPATH,
    "//div[contains(@class, 'usaha-card') and @data-id][1]//button[contains(@class, 'btn-tandai')]"
)


tandai_btn = wait.until(EC.element_to_be_clickable(btn_locator))  # nunggu clickable pakai locator [web:30][web:104]
driver.execute_script("arguments[0].scrollIntoView({block:'center'});", tandai_btn)
tandai_btn.click()
time.sleep(1)

# ======================
# 5. ISI USAHA
# ======================
# Ambil data baris pertama (strip NaN)
keberadaan = str(sbrdup["keberadaan_usaha"].iloc[4]).strip()
lat = str(sbrdup["latitude"].iloc[4]).strip()
lon = str(sbrdup["longitude"].iloc[4]).strip()

# 1) Klik & pilih dropdown #tt_hasil_gc
select = wait.until(EC.element_to_be_clickable((By.ID, "tt_hasil_gc")))
select.click()
driver.execute_script("arguments[0].value = arguments[1]; arguments[0].dispatchEvent(new Event('change'));", select, keberadaan)
time.sleep(1)

# 2-3) Enter otomatis via JS change event (gak perlu manual ENTER)

# 4) Isi latitude #tt_latitude_cek_user
lat_inp = wait.until(EC.visibility_of_element_located((By.ID, "tt_latitude_cek_user")))
lat_inp.clear()
lat_inp.send_keys(lat)

# 5) Isi longitude #tt_longitude_cek_user
lon_inp = wait.until(EC.visibility_of_element_located((By.ID, "tt_longitude_cek_user")))
lon_inp.clear()
lon_inp.send_keys(lon)

# Klik Save "TANDAI USAHA SUDAH DICEK"
save_btn = wait.until(EC.element_to_be_clickable((By.ID, "save-tandai-usaha-btn")))
driver.execute_script("arguments[0].scrollIntoView({block:'center'});", save_btn)
save_btn.click()

# Tunggu & klik SweetAlert OK
time.sleep(2)
ok_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.swal2-confirm")))
ok_btn.click()

# Atau JS kalau intercept
# driver.execute_script("document.querySelector('button.swal2-confirm').click();")
time.sleep(2)  # biar animasi tutup
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(1)


# Fungsi Isi Otomatis

In [6]:
def input_matcha(df):
    """
    Loop entri semua baris df (sbrdup) satu-satu.
    Skip NaN idsbr atau badge "Sudah GC".
    Retry baris saat error dengan refresh halaman.
    Refresh terus sampe berhasil, ga akan skip baris.
    """
    import time as tm
    
    total_rows = len(df)
    processed = 0
    skipped = 0
    
    idx = 0
    while idx < total_rows:
        start_time = tm.time()
        
        idsbr_raw = str(df["idsbr"].iloc[idx]).strip()
        if not idsbr_raw or pd.isna(df["idsbr"].iloc[idx]):
            print(f"Skip baris {idx}: IDSBR kosong")
            skipped += 1
            idx += 1
            continue
            
        print(f"\n▶ Baris {idx+1}/{total_rows} | IDSBR: {idsbr_raw}")
        
        try:
            # Scroll top
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(0.75)
            
            # CEK filter sudah terbuka atau belum (search-idsbr visible)
            wait = WebDriverWait(driver, 30, ignored_exceptions=(StaleElementReferenceException, NoSuchElementException))
            
            try:
                # Coba cek input IDSBR sudah visible atau belum
                inp = driver.find_element(By.ID, "search-idsbr")
                if not inp.is_displayed():
                    raise Exception("Input belum visible")
            except:
                # Kalau belum muncul, klik toggle filter
                wait.until(EC.element_to_be_clickable((By.ID, "toggle-filter"))).click()
                time.sleep(0.75)
            
            # Isi IDSBR
            inp = wait.until(EC.element_to_be_clickable((By.ID, "search-idsbr")))
            inp.click()
            inp.send_keys(Keys.CONTROL, "a")
            inp.send_keys(Keys.BACKSPACE)
            inp.send_keys(idsbr_raw)
            time.sleep(4)
            
            # Scroll bottom load card
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            
            # CEK BADGE "Sudah GC" - kalau ada skip
            try:
                driver.find_element(By.CSS_SELECTOR, "span.gc-badge")
                elapsed = tm.time() - start_time
                print(f"✓ Skip: Badge GC ditemukan ({elapsed:.1f}s)")
                skipped += 1
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(1)
                idx += 1
                continue
            except:
                pass
            
            # Klik card expand
            usaha = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.usaha-card")))
            usaha.click()
            time.sleep(0.75)
            
            # Scroll & klik Tandai
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.75)

            # CEK BADGE DUPLIKAT - kalau ada skip
            try:
                driver.find_element(By.CSS_SELECTOR, "div.duplikat-notice-title")
                elapsed = tm.time() - start_time
                print(f"⚠ Skip: Data Duplikat ({elapsed:.1f}s)")
                skipped += 1
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(0.75)
                idx += 1
                continue
            except:
                pass

            btn_locator = (By.XPATH, "//div[contains(@class, 'usaha-card') and @data-id][1]//button[contains(@class, 'btn-tandai')]")
            tandai_btn = wait.until(EC.element_to_be_clickable(btn_locator))
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", tandai_btn)
            tandai_btn.click()
            time.sleep(0.75)
            
            # Isi form
            keberadaan = str(df["keberadaan_usaha"].iloc[idx]).strip()
            lat = str(df["latitude"].iloc[idx]).strip()
            lon = str(df["longitude"].iloc[idx]).strip()
            
            select = wait.until(EC.element_to_be_clickable((By.ID, "tt_hasil_gc")))
            select.click()
            driver.execute_script("arguments[0].value = arguments[1]; arguments[0].dispatchEvent(new Event('change'));", select, keberadaan)
            time.sleep(0.75)
            
            # Isi lat/lon HANYA kalau keberadaan_usaha != 4
            if keberadaan != "4":
                lat_inp = wait.until(EC.visibility_of_element_located((By.ID, "tt_latitude_cek_user")))
                lat_inp.clear()
                lat_inp.send_keys(lat)
                
                lon_inp = wait.until(EC.visibility_of_element_located((By.ID, "tt_longitude_cek_user")))
                lon_inp.clear()
                lon_inp.send_keys(lon)
            else:
                print("  ⚠ Keberadaan=4, skip isi lat/lon")
            
            save_btn = wait.until(EC.element_to_be_clickable((By.ID, "save-tandai-usaha-btn")))
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", save_btn)
            save_btn.click()
            
            time.sleep(1.5)
            ok_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.swal2-confirm")))
            ok_btn.click()
            time.sleep(0.75)
            
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(0.75)
            processed += 1
            elapsed = tm.time() - start_time
            print(f"✅ Selesai ({elapsed:.1f}s)")
            idx += 1
            
        except Exception as e:
            elapsed = tm.time() - start_time
            print(f"❌ Error baris {idx+1} ({elapsed:.1f}s): {str(e)[:100]}")
            print("🔄 Refresh halaman & retry baris ini...")
            
            # Infinite retry loop sampe berhasil refresh
            refresh_success = False
            while not refresh_success:
                for attempt in range(3):
                    try:
                        driver.get("https://matchapro.web.bps.go.id/dirgc")
                        time.sleep(2.25)
                        refresh_success = True
                        break
                    except Exception as refresh_err:
                        print(f"  ⚠ Refresh gagal attempt {attempt+1}/3: {str(refresh_err)[:60]}")
                        time.sleep(1.5)
                
                if not refresh_success:
                    print("  ⏳ Refresh gagal 3x, sleep 20 detik & retry...")
                    time.sleep(15)
    
    print(f"\n✔ Processed: {processed}, Skipped: {skipped}/{total_rows}")
    return df


# ISI

In [11]:
input_matcha(sbrdup)


▶ Baris 1/13000 | IDSBR: 27729
✓ Skip: Badge GC ditemukan (7.2s)

▶ Baris 2/13000 | IDSBR: 234899
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 3/13000 | IDSBR: 94412460
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 4/13000 | IDSBR: 63410008
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 5/13000 | IDSBR: 94417213
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 6/13000 | IDSBR: 94421387
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 7/13000 | IDSBR: 94440520
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 8/13000 | IDSBR: 94652836
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 9/13000 | IDSBR: 94431335
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 10/13000 | IDSBR: 95333034
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 11/13000 | IDSBR: 94725724
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 12/13000 | IDSBR: 97317653
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 13/13000 | IDSBR: 97317660
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 14/13000 | IDSBR: 97318360
✓ Skip: Badge GC ditemukan (6.4s)

▶ Baris 15/13000 | IDSBR: 9731764

,nomor,Petugas,idsbr,nama_usaha,nama_komersial_usaha,alamat,nama_sls,kodepos,nomor_telepon,nomor_whatsapp,email,website,latitude,longitude,keberadaan_usaha,kdprov,kdkab,kdkec,kddesa,nmkec,nmdesa,kbli,sumber_profiling,catatan_profiling,norm_nama_usaha,hasil_maps,flag_dup
0,1,"Gilang Wahyu Prasetyo, S.Tr.Stat",27729,BUKIT MAKMUR MANDIRI UTAMA PT,BUKIT MAKMUR MANDIRI UTAMA PT,JALAN HAULING ROAD ADARO KM 73 P-HW T300 DESA ...,NaN,71662.0,82244789456,NaN,NaN,NaN,-2.236413065,115.4710211,1,63,9,50.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"Gilang Wahyu Prasetyo, S.Tr.Stat",234899,KOPKAR PERTAMINA,Koperasi karmina pertamina,"Komp. Pertamina Murung Pudak, Kec. Murung Pudak",NaN,71571.0,NaN,NaN,NaN,NaN,-2.144423793,115.3982164,1,63,9,70.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"Gilang Wahyu Prasetyo, S.Tr.Stat",94412460,"ADIRA DINAMIKA MULTI FINANCE TBK, PT",NaN,Jl. Ir. P. H. M. Noor Bundaran Obor RT RW 005,Adira Finance Cab Tanjung,71571.0,5262022994,NaN,ADIRA.FINANCE@ADMIN.CO.ID,NaN,-2.172343264,115.4134518,1,63,9,70.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"Gilang Wahyu Prasetyo, S.Tr.Stat",63410008,PT Air Minum Tabalong Bersinar (AMTB),PDAM KAB TABALONG,JL KAMBOJA NO 1 tanjung,NaN,71513.0,0526-2021112,NaN,umum@pdamtabalong.co.id,NaN,-2.163336,115.38332,1,63,9,60.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"Gilang Wahyu Prasetyo, S.Tr.Stat",94417213,"SAPTAINDRA SEJATI, PT","SIS, PT",JL MINING OFFICE ADARO TUTUPANCOAL MINING,NaN,71571.0,81346403252,NaN,wahyu.jatisaptaindra@gmail.com,NaN,-2.254441186,115.434249,1,63,9,50.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,12996,"Gilang Wahyu Prasetyo, S.Tr.Stat",15614254,PEDAGANG IKAN <MAHDI> <MAHDI>,NaN,KELILING DI UPAU,RT 003-[00],NaN,NaN,NaN,NaN,NaN,-2.3162943,115.3069673,1,63,9,20.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12996,12997,"Gilang Wahyu Prasetyo, S.Tr.Stat",15614629,sevice elektronik <RIDWAN>,NaN,BOUGENVILLE,RT 019-[00],NaN,NaN,85248567567,NaN,NaN,-2.1615392,115.3983625,1,63,9,70.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12997,12998,"Gilang Wahyu Prasetyo, S.Tr.Stat",15609662,RUMAH MAKAN AYA <IWAN> <IWAN>,NaN,JL.PHM NOOR SULINGAN RT.06,RT 006-[00],NaN,NaN,81250529078,NaN,NaN,-2.1692333,115.38213,1,63,9,70.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12998,12999,"Gilang Wahyu Prasetyo, S.Tr.Stat",15610306,BELLY MOTOR <JUMADI>,NaN,Hikun RT 008,RT 008-[00],NaN,NaN,81348042122,NaN,NaN,-2.1365422,115.3921316,1,63,9,60.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Kalo stuck di baris tertentu, pake ini untuk menghapus baris2 pertama yang berhasil dientri biar ga ngulang dari awal. ganti angkanya sesuai jumlah baris yang udah kamu entri di bagian iloc[xx:] (ganti xx nya) 

In [ ]:
# Hapus 39 baris pertama (index 0-38)
sbrdup = sbrdup.iloc[166:].reset_index(drop=True)
print(f"Sekarang {len(sbrdup)} baris tersisa")
#sbrdup.to_excel(r"C:\Users\tiomultazem\Downloads\sbrdup.xlsx", index=False)
